In [ ]:
import madge from 'npm:madge';

const dependenciesBase = await madge(Deno.cwd() + '/opencrvs-core/packages/client/src/index.tsx', {
  tsConfig: Deno.cwd() + '/opencrvs-core/packages/client/tsconfig.json',
  baseDir: Deno.cwd() + '/opencrvs-core',
}).then((res) => res.obj())

const dependenciesHead = await madge(Deno.cwd() + '/opencrvs-core_head/packages/client/src/index.tsx', {
  tsConfig: Deno.cwd() + '/opencrvs-core_head/packages/client/tsconfig.json',
  baseDir: Deno.cwd() + '/opencrvs-core_head',
}).then((res) => res.obj())


const foo = await madge(Deno.cwd() + '/opencrvs-core_head/packages/client/src/index.tsx', {
  tsConfig: Deno.cwd() + '/opencrvs-core_head/packages/client/tsconfig.json',
  baseDir: Deno.cwd() + '/opencrvs-core_head',
})
foo


In [ ]:
const files = Array.from(new Set(Object.keys(dependenciesBase).concat(Object.keys(dependenciesHead))))

files.forEach((file) => {
  if (dependenciesBase[file] && dependenciesHead[file]) {
    const removedDependencies = dependenciesBase[file].filter((dep) => dependenciesHead[file].indexOf(dep) === -1)
    const addedDependencies = dependenciesHead[file].filter((dep) => dependenciesBase[file].indexOf(dep) === -1)

    if(removedDependencies.length > 0) {
      console.log(`${file}:\n   - ${removedDependencies.join('\n   - ')}`)
    }
    if(addedDependencies.length > 0) {
      console.log(`   + ${addedDependencies.join('\n   + ')}`)
    }

    // dependenciesBase[file].forEach((dep) => {
    //   if (dependenciesHead[file].indexOf(dep) === -1) {
    //     console.log(`${dep} is no longer a dependency of ${file}`)
    //   }
    // })
  }
})

In [112]:
import ts from "typescript";
import * as path from "https://deno.land/std/path/mod.ts";

function getImportsWithResolvedPaths(
  filePath: string,
  tsConfigPath: string,
  basePath: string
): { importedSymbols: string[]; fromFile: string }[] {
  const config = ts.readConfigFile(tsConfigPath, ts.sys.readFile);
  if (!config.config) {
    throw new Error(`Invalid tsconfig file: ${tsConfigPath}`);
  }

  const parsedConfig = ts.parseJsonConfigFileContent(
    config.config,
    ts.sys,
    path.dirname(tsConfigPath)
  );

  const compilerOptions = parsedConfig.options;
  const host = ts.createCompilerHost(compilerOptions);
  const program = ts.createProgram([filePath], compilerOptions, host);
  const sourceFile = program.getSourceFile(filePath);

  if (!sourceFile) {
    throw new Error(`Failed to read file: ${filePath}`);
  }

  const imports: { importedSymbols: string[]; fromFile: string }[] = [];

  sourceFile.forEachChild((node) => {
    if (ts.isImportDeclaration(node) && node.moduleSpecifier) {
      const moduleSpecifier = (node.moduleSpecifier as ts.StringLiteral).text;
      const importedSymbols: string[] = [];

      if (node.importClause) {
        const { namedBindings, name } = node.importClause;

        // Default import
        if (name) {
          importedSymbols.push(name.text);
        }

        // Named imports and namespace imports
        if (namedBindings) {
          if (ts.isNamedImports(namedBindings)) {
            namedBindings.elements.forEach((element) => {
              importedSymbols.push(element.name.text);
            });
          } else if (ts.isNamespaceImport(namedBindings)) {
            importedSymbols.push(`* as ${namedBindings.name.text}`);
          }
        }
      }

      const resolvedModule = ts.resolveModuleName(
        moduleSpecifier,
        filePath,
        compilerOptions,
        host
      );
      if (
        !resolvedModule.resolvedModule ||
        resolvedModule.resolvedModule.isExternalLibraryImport
      ) {
        return;
      }

      const resolvedFileName =
        resolvedModule.resolvedModule.resolvedFileName || moduleSpecifier;

      imports.push({
        importedSymbols,
        fromFile: resolvedFileName.replace(basePath, ''),
        resolvedModule: resolvedModule.resolvedModule,
      });
    }
  });

  return imports;
}


In [107]:
async function getAllFiles() {
  const prUrl =
    "https://api.github.com/repos/opencrvs/opencrvs-core/pulls/7301";

  const headers = {
    Accept: "application/vnd.github.v3+json",
  };

  let allFiles = [];
  let page = 1;
  let hasMore = true;

  while (hasMore) {
    const response = await fetch(`${prUrl}/files?per_page=100&page=${page}`, {
      headers,
    });

    if (!response.ok) {
      throw new Error(`GitHub API error: ${response.status} ${response.statusText}`);
    }

    const files = await response.json();
    allFiles = allFiles.concat(files);

    if (files.length < 100) {
      hasMore = false; // No more pages
    } else {
      page += 1;
    }
  }

  return allFiles;
}

In [5]:
const files = await getAllFiles()

In [116]:
const typescriptFiles = files
.filter(file => file.status === 'modified')
.filter((file) => file.filename.endsWith(".ts") || file.filename.endsWith(".tsx"))
.slice(20, 40);



const symbolsPerFile = typescriptFiles.map((file) => {
  return {
    filename: file.filename,
    oldSymbols: getImportsWithResolvedPaths(
      Deno.cwd() + '/opencrvs-core/' + file.filename,
      Deno.cwd() + '/opencrvs-core/packages/client/tsconfig.json',
      Deno.cwd() + '/opencrvs-core'
    ),
    newSymbols: getImportsWithResolvedPaths(
      Deno.cwd() + '/opencrvs-core_head/' + file.filename,
      Deno.cwd() + '/opencrvs-core_head/packages/client/tsconfig.json',
      Deno.cwd() + '/opencrvs-core_head'
    ),
  };
})

In [ ]:
symbolsPerFile.flatMap(({ oldSymbols, newSymbols, filename }) => {
  const addedSymbols = newSymbols.flatMap((newSymbol) => {
    return newSymbol.importedSymbols
      .filter((symbol) => {
        const existingImportStatement = oldSymbols.some(
          (oldSymbol) =>
            oldSymbol.fromFile ===
            newSymbol.fromFile
        );
        if (!existingImportStatement) {
          return true;
        }

        return !oldSymbols
          .find((oldSymbol) => oldSymbol.fromFile === newSymbol.fromFile)
          .importedSymbols.includes(symbol);
      })
      .map((symbol) => ({
        filename,
        status: "added",
        symbol: symbol.startsWith("* as ") ? null : symbol,
        fromFile: newSymbol.fromFile,
      }));
  });
  return addedSymbols;
});
